<a href="https://colab.research.google.com/github/HChandeepa/Reddit-posts-on-the-Israel-Palestine-War-War/blob/Master/Reddit_Israel_and__Gaza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/Israel_Gaza_War_Reddit_Data_Optimized_with_Comments1 - Copy.csv')

In [3]:
df.head()

,Post_Title,Comment_Text,Comment_Upvotes,Comment_Created_Date
0,Israel resumes water supply to southern Gaza a...,As per CNN\n\nIsraeli Prime Minister Benjamin ...,4628.0,10/15/2023 17:52
1,Israel resumes water supply to southern Gaza a...,What’s the long game here?(3ish weeks) does Is...,703.0,10/15/2023 16:54
2,Israel resumes water supply to southern Gaza a...,Good job to U.S. diplomats for changing Isreal...,5684.0,10/15/2023 16:24
3,Israel resumes water supply to southern Gaza a...,The US is in a crucial position to advance the...,1805.0,10/15/2023 16:44
4,Israel resumes water supply to southern Gaza a...,Everybody hates the United States until it’s t...,230.0,10/15/2023 20:14


In [4]:
df.shape

(15191, 4)

## Task 2: Text Preprocessing and Tokenization


In [19]:
import spacy
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.util import ngrams

In [28]:
# Load SpaCy model
nlp = spacy.load('en_core_web_sm')

def preprocess_with_spacy(text):
    if isinstance(text, str):  # Check if the input is a string
        doc = nlp(text)  # Process text with SpaCy
        tokens = [
            token.lemma_  # Lemmatize each word
            for token in doc
            if not token.is_stop and token.is_alpha  # Remove stopwords and special characters
        ]
        return tokens  # Return tokens as a list
    return []  # Return an empty list for missing or invalid input

In [29]:
# Apply preprocessing and tokenization
df['Tokenized_Text'] = df['Comment_Text'].apply(preprocess_with_spacy)

In [30]:
print(df[['Comment_Text', 'Tokenized_Text']].head())

                                        Comment_Text  \
0  As per CNN\n\nIsraeli Prime Minister Benjamin ...   
1  What’s the long game here?(3ish weeks) does Is...   
2  Good job to U.S. diplomats for changing Isreal...   
3  The US is in a crucial position to advance the...   
4  Everybody hates the United States until it’s t...   

                                      Tokenized_Text  
0  [CNN, israeli, Prime, Minister, Benjamin, Neta...  
1  [long, game, week, Israel, obliterate, norther...  
2        [good, job, diplomat, change, Isreal, mind]  
3  [crucial, position, advance, humanitarian, cor...  
4  [everybody, hate, United, States, time, United...  


In [31]:
print("Missing values in Preprocessed_Comment_Text:", df['Tokenized_Text'].isnull().sum())

Missing values in Preprocessed_Comment_Text: 0


In [32]:
print(df['Tokenized_Text'].dropna().head())

0    [CNN, israeli, Prime, Minister, Benjamin, Neta...
1    [long, game, week, Israel, obliterate, norther...
2          [good, job, diplomat, change, Isreal, mind]
3    [crucial, position, advance, humanitarian, cor...
4    [everybody, hate, United, States, time, United...
Name: Tokenized_Text, dtype: object


In [33]:
for i in range(5):  # Adjust range for more samples
    original = df['Comment_Text'].iloc[i]
    processed = df['Tokenized_Text'].iloc[i]
    print(f"Original: {original}")
    print(f"Processed: {processed}")
    print("-----------")

Original: As per CNN

Israeli Prime Minister Benjamin Netanyahu’s office told CNN Sunday Israel has restored water to southern Gaza, but the director of the Palestinian Water Authority said he could not be sure it was true, because electricity has not been restored.

“I cannot confirm this at the moment because our water stations in Khan Younis, Gaza City, and central Gaza are not in a condition to receive and distribute water to the people,” Munther Shublaq, the director of the Water Authority in Gaza, told CNN.
“To enable the stations to receive and distribute water, we need electricity, which is currently cut off. The alternative is fuel to power the generator, which we also do not have. There may be pressure to restore the water supply, but I cannot confirm anything until the stations are ready.”
Processed: ['CNN', 'israeli', 'Prime', 'Minister', 'Benjamin', 'Netanyahu', 'office', 'tell', 'CNN', 'Sunday', 'Israel', 'restore', 'water', 'southern', 'Gaza', 'director', 'Palestinian', 

In [34]:
# Intialize Stemmer
Stemmer = PorterStemmer()

In [35]:
def stem_with_nltk(tokens):
    if isinstance(tokens, list):  # Ensure the input is a list
        return [Stemmer.stem(token) for token in tokens]
    return []  # Return an empty list for invalid input


In [36]:
df['Stemmed_Text'] = df['Tokenized_Text'].apply(stem_with_nltk)

In [37]:
print(df[['Comment_Text', 'Tokenized_Text', 'Stemmed_Text']].head())

                                        Comment_Text  \
0  As per CNN\n\nIsraeli Prime Minister Benjamin ...   
1  What’s the long game here?(3ish weeks) does Is...   
2  Good job to U.S. diplomats for changing Isreal...   
3  The US is in a crucial position to advance the...   
4  Everybody hates the United States until it’s t...   

                                      Tokenized_Text  \
0  [CNN, israeli, Prime, Minister, Benjamin, Neta...   
1  [long, game, week, Israel, obliterate, norther...   
2        [good, job, diplomat, change, Isreal, mind]   
3  [crucial, position, advance, humanitarian, cor...   
4  [everybody, hate, United, States, time, United...   

                                        Stemmed_Text  
0  [cnn, isra, prime, minist, benjamin, netanyahu...  
1  [long, game, week, israel, obliter, northern, ...  
2         [good, job, diplomat, chang, isreal, mind]  
3  [crucial, posit, advanc, humanitarian, corrido...  
4  [everybodi, hate, unit, state, time, unit, sta..

In [39]:
# Generate n-grams
def generate_ngrams(tokens, n):
    return list(ngrams(tokens, n))

df['Bigrams'] = df['Stemmed_Text'].apply(lambda tokens: generate_ngrams(tokens, 2))
df['Trigrams'] = df['Stemmed_Text'].apply(lambda tokens: generate_ngrams(tokens, 3))

In [40]:
# Display the original tokens and n-grams for verification
for i in range(5):  # Adjust the range to inspect more rows
    print(f"Row {i + 1}:")
    print(f"Original Tokens: {df['Stemmed_Text'].iloc[i]}")
    print(f"Bigrams: {df['Bigrams'].iloc[i]}")
    print(f"Trigrams: {df['Trigrams'].iloc[i]}")
    print("-" * 50)


Row 1:
Original Tokens: ['cnn', 'isra', 'prime', 'minist', 'benjamin', 'netanyahu', 'offic', 'tell', 'cnn', 'sunday', 'israel', 'restor', 'water', 'southern', 'gaza', 'director', 'palestinian', 'water', 'author', 'say', 'sure', 'true', 'electr', 'restor', 'confirm', 'moment', 'water', 'station', 'khan', 'youni', 'gaza', 'citi', 'central', 'gaza', 'condit', 'receiv', 'distribut', 'water', 'peopl', 'munther', 'shublaq', 'director', 'water', 'author', 'gaza', 'tell', 'cnn', 'enabl', 'station', 'receiv', 'distribut', 'water', 'need', 'electr', 'current', 'cut', 'altern', 'fuel', 'power', 'gener', 'pressur', 'restor', 'water', 'suppli', 'confirm', 'station', 'readi']
Bigrams: [('cnn', 'isra'), ('isra', 'prime'), ('prime', 'minist'), ('minist', 'benjamin'), ('benjamin', 'netanyahu'), ('netanyahu', 'offic'), ('offic', 'tell'), ('tell', 'cnn'), ('cnn', 'sunday'), ('sunday', 'israel'), ('israel', 'restor'), ('restor', 'water'), ('water', 'southern'), ('southern', 'gaza'), ('gaza', 'director'), 

In [41]:
print(df[['Stemmed_Text', 'Bigrams', 'Trigrams']].head())

                                        Stemmed_Text  \
0  [cnn, isra, prime, minist, benjamin, netanyahu...   
1  [long, game, week, israel, obliter, northern, ...   
2         [good, job, diplomat, chang, isreal, mind]   
3  [crucial, posit, advanc, humanitarian, corrido...   
4  [everybodi, hate, unit, state, time, unit, sta...   

                                             Bigrams  \
0  [(cnn, isra), (isra, prime), (prime, minist), ...   
1  [(long, game), (game, week), (week, israel), (...   
2  [(good, job), (job, diplomat), (diplomat, chan...   
3  [(crucial, posit), (posit, advanc), (advanc, h...   
4  [(everybodi, hate), (hate, unit), (unit, state...   

                                            Trigrams  
0  [(cnn, isra, prime), (isra, prime, minist), (p...  
1  [(long, game, week), (game, week, israel), (we...  
2  [(good, job, diplomat), (job, diplomat, chang)...  
3  [(crucial, posit, advanc), (posit, advanc, hum...  
4  [(everybodi, hate, unit), (hate, unit, state),..

In [42]:
# Convert n-grams to readable strings for easier verification
df['Readable_Bigrams'] = df['Bigrams'].apply(lambda bigrams: [' '.join(bigram) for bigram in bigrams])
df['Readable_Trigrams'] = df['Trigrams'].apply(lambda trigrams: [' '.join(trigram) for trigram in trigrams])

# Display the readable n-grams
print(df[['Stemmed_Text', 'Readable_Bigrams', 'Readable_Trigrams']].head())

                                        Stemmed_Text  \
0  [cnn, isra, prime, minist, benjamin, netanyahu...   
1  [long, game, week, israel, obliter, northern, ...   
2         [good, job, diplomat, chang, isreal, mind]   
3  [crucial, posit, advanc, humanitarian, corrido...   
4  [everybodi, hate, unit, state, time, unit, sta...   

                                    Readable_Bigrams  \
0  [cnn isra, isra prime, prime minist, minist be...   
1  [long game, game week, week israel, israel obl...   
2  [good job, job diplomat, diplomat chang, chang...   
3  [crucial posit, posit advanc, advanc humanitar...   
4  [everybodi hate, hate unit, unit state, state ...   

                                   Readable_Trigrams  
0  [cnn isra prime, isra prime minist, prime mini...  
1  [long game week, game week israel, week israel...  
2  [good job diplomat, job diplomat chang, diplom...  
3  [crucial posit advanc, posit advanc humanitari...  
4  [everybodi hate unit, hate unit state, unit st..

After using stemming some words important parts are dropped. Therefore , I am going to use lemmatization now.

In [43]:
def preprocess_with_spacy(text):
    if isinstance(text, str):
        doc = nlp(text)
        tokens = [
            token.lemma_  # Use SpaCy's lemmatization
            for token in doc
            if not token.is_stop and token.is_alpha  # Remove stopwords and punctuation
        ]
        return tokens  # Return a list of lemmatized tokens
    return []

In [44]:
# Apply preprocessing and tokenization
df['Tokenized_Text'] = df['Comment_Text'].apply(preprocess_with_spacy)

In [45]:
print(df[['Comment_Text', 'Tokenized_Text']].head())

                                        Comment_Text  \
0  As per CNN\n\nIsraeli Prime Minister Benjamin ...   
1  What’s the long game here?(3ish weeks) does Is...   
2  Good job to U.S. diplomats for changing Isreal...   
3  The US is in a crucial position to advance the...   
4  Everybody hates the United States until it’s t...   

                                      Tokenized_Text  
0  [CNN, israeli, Prime, Minister, Benjamin, Neta...  
1  [long, game, week, Israel, obliterate, norther...  
2        [good, job, diplomat, change, Isreal, mind]  
3  [crucial, position, advance, humanitarian, cor...  
4  [everybody, hate, United, States, time, United...  


In [46]:
print("Missing values in Preprocessed_Comment_Text:", df['Tokenized_Text'].isnull().sum())

Missing values in Preprocessed_Comment_Text: 0


In [47]:
print(df['Tokenized_Text'].dropna().head())

0    [CNN, israeli, Prime, Minister, Benjamin, Neta...
1    [long, game, week, Israel, obliterate, norther...
2          [good, job, diplomat, change, Isreal, mind]
3    [crucial, position, advance, humanitarian, cor...
4    [everybody, hate, United, States, time, United...
Name: Tokenized_Text, dtype: object


In [48]:
for i in range(5):  # Adjust range for more samples
    original = df['Comment_Text'].iloc[i]
    processed = df['Tokenized_Text'].iloc[i]
    print(f"Original: {original}")
    print(f"Processed: {processed}")
    print("-----------")

Original: As per CNN

Israeli Prime Minister Benjamin Netanyahu’s office told CNN Sunday Israel has restored water to southern Gaza, but the director of the Palestinian Water Authority said he could not be sure it was true, because electricity has not been restored.

“I cannot confirm this at the moment because our water stations in Khan Younis, Gaza City, and central Gaza are not in a condition to receive and distribute water to the people,” Munther Shublaq, the director of the Water Authority in Gaza, told CNN.
“To enable the stations to receive and distribute water, we need electricity, which is currently cut off. The alternative is fuel to power the generator, which we also do not have. There may be pressure to restore the water supply, but I cannot confirm anything until the stations are ready.”
Processed: ['CNN', 'israeli', 'Prime', 'Minister', 'Benjamin', 'Netanyahu', 'office', 'tell', 'CNN', 'Sunday', 'Israel', 'restore', 'water', 'southern', 'Gaza', 'director', 'Palestinian', 

In [49]:
# Generate n-grams
df['Bigrams'] = df['Tokenized_Text'].apply(lambda tokens: generate_ngrams(tokens, 2))
df['Trigrams'] = df['Tokenized_Text'].apply(lambda tokens: generate_ngrams(tokens, 3))

In [52]:
# Display the original tokens and n-grams for verification
for i in range(5):  # Adjust the range to inspect more rows
    print(f"Row {i + 1}:")
    print(f"Original Tokens: {df['Tokenized_Text'].iloc[i]}")
    print(f"Bigrams: {df['Bigrams'].iloc[i]}")
    print(f"Trigrams: {df['Trigrams'].iloc[i]}")
    print("-" * 50)


Row 1:
Original Tokens: ['CNN', 'israeli', 'Prime', 'Minister', 'Benjamin', 'Netanyahu', 'office', 'tell', 'CNN', 'Sunday', 'Israel', 'restore', 'water', 'southern', 'Gaza', 'director', 'Palestinian', 'Water', 'Authority', 'say', 'sure', 'true', 'electricity', 'restore', 'confirm', 'moment', 'water', 'station', 'Khan', 'Younis', 'Gaza', 'City', 'central', 'Gaza', 'condition', 'receive', 'distribute', 'water', 'people', 'Munther', 'Shublaq', 'director', 'Water', 'Authority', 'Gaza', 'tell', 'CNN', 'enable', 'station', 'receive', 'distribute', 'water', 'need', 'electricity', 'currently', 'cut', 'alternative', 'fuel', 'power', 'generator', 'pressure', 'restore', 'water', 'supply', 'confirm', 'station', 'ready']
Bigrams: [('CNN', 'israeli'), ('israeli', 'Prime'), ('Prime', 'Minister'), ('Minister', 'Benjamin'), ('Benjamin', 'Netanyahu'), ('Netanyahu', 'office'), ('office', 'tell'), ('tell', 'CNN'), ('CNN', 'Sunday'), ('Sunday', 'Israel'), ('Israel', 'restore'), ('restore', 'water'), ('wate

In [53]:
print(df[['Tokenized_Text', 'Bigrams', 'Trigrams']].head())

                                      Tokenized_Text  \
0  [CNN, israeli, Prime, Minister, Benjamin, Neta...   
1  [long, game, week, Israel, obliterate, norther...   
2        [good, job, diplomat, change, Isreal, mind]   
3  [crucial, position, advance, humanitarian, cor...   
4  [everybody, hate, United, States, time, United...   

                                             Bigrams  \
0  [(CNN, israeli), (israeli, Prime), (Prime, Min...   
1  [(long, game), (game, week), (week, Israel), (...   
2  [(good, job), (job, diplomat), (diplomat, chan...   
3  [(crucial, position), (position, advance), (ad...   
4  [(everybody, hate), (hate, United), (United, S...   

                                            Trigrams  
0  [(CNN, israeli, Prime), (israeli, Prime, Minis...  
1  [(long, game, week), (game, week, Israel), (we...  
2  [(good, job, diplomat), (job, diplomat, change...  
3  [(crucial, position, advance), (position, adva...  
4  [(everybody, hate, United), (hate, United, Sta..

In [54]:
# Convert n-grams to readable strings for easier verification
df['Readable_Bigrams'] = df['Bigrams'].apply(lambda bigrams: [' '.join(bigram) for bigram in bigrams])
df['Readable_Trigrams'] = df['Trigrams'].apply(lambda trigrams: [' '.join(trigram) for trigram in trigrams])

# Display the readable n-grams
print(df[['Tokenized_Text', 'Readable_Bigrams', 'Readable_Trigrams']].head())

                                      Tokenized_Text  \
0  [CNN, israeli, Prime, Minister, Benjamin, Neta...   
1  [long, game, week, Israel, obliterate, norther...   
2        [good, job, diplomat, change, Isreal, mind]   
3  [crucial, position, advance, humanitarian, cor...   
4  [everybody, hate, United, States, time, United...   

                                    Readable_Bigrams  \
0  [CNN israeli, israeli Prime, Prime Minister, M...   
1  [long game, game week, week Israel, Israel obl...   
2  [good job, job diplomat, diplomat change, chan...   
3  [crucial position, position advance, advance h...   
4  [everybody hate, hate United, United States, S...   

                                   Readable_Trigrams  
0  [CNN israeli Prime, israeli Prime Minister, Pr...  
1  [long game week, game week Israel, week Israel...  
2  [good job diplomat, job diplomat change, diplo...  
3  [crucial position advance, position advance hu...  
4  [everybody hate United, hate United States, Un..